<a href="https://colab.research.google.com/github/black-beard2002/github-workshop-pages/blob/main/chatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:

from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\n         "context_set": ""\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye"],\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\n        },\n        {"tag": "hours",\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\n        },\n        {"tag": "location",\n         "patterns": ["What is your location?", "Where are 

In [10]:

# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [11]:

words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [12]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

31 documents
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 unique stemmed words ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


In [15]:
# Create training data
training = []
output = []
output_empty = [0] * len(classes)

# Create training set, bag of words for each sentence
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # Debugging: Check lengths
    print("Length of bag:", len(bag))
    print("Length of output_row:", len(output_row))

    training.append([bag, output_row])

# Separate bag and output_row into train_x and train_y
train_x = [x[0] for x in training]
train_y = [x[1] for x in training]

# Convert train_x and train_y into NumPy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)


Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output_row: 9
Length of bag: 57
Length of output

In [17]:

# resetting underlying graph data
tf.compat.v1.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.02366 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.02366 - acc: 1.0000 -- iter: 24/31
Training Step: 4000  | total loss: 0.02315 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.02315 - acc: 1.0000 -- iter: 31/31
--


In [18]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [19]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [20]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [21]:
# load the saved model
model.load('./model.tflearn')

In [22]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [23]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [24]:
classify('What are you hours of operation?')

[('hours', 0.9946743)]

In [25]:
response('What are you hours of operation?')

We're open every day 9am-9pm


In [26]:
response('What is menu for today?')

Today's special is Chicken Tikka


In [27]:
response('What is menu for today?')

Today's special is Chicken Tikka


In [28]:
response('Do you accept Credit Card?')

We accept most major credit cards


In [29]:
response('Do you accept Credit Card?')

We accept most major credit cards


In [30]:
response('Do you accept Credit Card?')

We accept VISA, Mastercard and AMEX


In [31]:
response('Where can we locate you?')

We are on the intersection of London Alley and Bridge Avenue.


In [32]:
response('That is helpful')

Happy to help!


In [33]:
response('That is helpful')

My pleasure


In [34]:
response('That is helpful')

My pleasure


In [35]:
response('That is helpful')

Happy to help!


In [36]:
response('Bye')

Have a nice day


In [37]:
#Adding some context to the conversation i.e. Contexualization for altering question and intents etc.
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [38]:
response('Can you please let me know the delivery options?')

Yes, we provide home delivery through UBER Eats and Zomato?


In [39]:
response('What is menu for today?')

Today's special is Chicken Tikka


In [40]:
response('What is menu for today?')

Today's special is Chicken Tikka


In [41]:
response('What is menu for today?')

Today's special is Chicken Tikka


In [42]:
response('What is menu for today?')

Our speciality for today is Chicken Tikka


In [43]:
response('What is menu for today?')

Today's special is Chicken Tikka


In [44]:
response('What is menu for today?')

Our speciality for today is Chicken Tikka


In [45]:
response('What is menu for today?')

Today's special is Chicken Tikka


In [46]:
response('What is menu for today?')

Our speciality for today is Chicken Tikka


In [47]:
context

{'123': 'food'}

In [48]:
response("Hi there!", show_details=True)

context: 
tag: greeting
Good to see you again


In [49]:

response('What is menu for today?')

Today's special is Chicken Tikka


In [50]:

response('who are you?')

Hi there, how can I help?


In [51]:

response('What is your name?')

Our Address is: 1000 Bridge Avenue, London EC3N 4AJ, UK


In [52]:

response('how are you?')

Hello, thanks for visiting


In [53]:

response('What is the full menu?')

In [54]:

response('What is menu for today?')

Our speciality for today is Chicken Tikka
